In [1]:
import paramiko

ip = "172.16.16.122"
user = "root"
pw = "asd123!@"

## server ##
server_ip = "172.16.16.122"
server_user = "root"
server_pw = "asd123!@"
server_iqn = "iqn.2025-11.local.r1:disk1"

## client ##
client_ip = "172.16.16.25"
client_user = "root"
client_pw = "asd123!@"
client_os_type = "R"
client_iqn = "iqn.1994-05.com.redhat:64d3173ff775"


def cc(cmd):
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd, get_pty=True)  # ← 여기가 핵심
    out = stdout.read().decode()
    err = stderr.read().decode()
    print(out if out else err)
    cli.close()

def change_remote_cc_from_server_to_client():
    global ip
    global user
    global pw
    ip = client_ip
    user = client_user
    pw = client_pw

def change_remote_cc_from_client_to_server():
    global ip
    global user
    global pw
    ip = server_ip
    user = server_user
    pw = server_pw

def iscsi_server_install():
    print("install을 시작합니다.")
    cc("dnf update -y && dnf upgrade -y")
    cc("dnf install targetcli -y")
    cc("systemctl stop firewalld")
    cc("dnf install -y expect")

def iscsi_server_targetcli_start() :
    print("targetcli 시작합니다.")
    cc("systemctl enable --now target;")

def iscsi_client_install_start() :
    global client_os_type
    global client_iqn

    change_remote_cc_from_server_to_client()
    
    client_os_type = input("client가 Rocky인 경우 R , Ubuntu인 경우 U를 입력해주세요")
    
    commands = "" 
    if(client_os_type == "R") :
        print("client : Rocky")
        commands = """
sudo dnf install -y iscsi-initiator-utils
sudo systemctl enable --now iscsid
cat /etc/iscsi/initiatorname.iscsi
        """
        #cc("ls -l")
        cc(commands)
        
        client_iqn = input("위에서 확인한 client의 iqn을 복사해주세요. ex. iqn.1994-05.com.redhat:30eb3450bd10 :")
        
#     if(client_os_type == "U") :
#         commands = """
# apt update && apt upgrade -y
# apt install targetcli-fb -y
# systemctl stop ufw
# systemctl enable --now target
# systemctl status target
#         """
    

def iscsi_server_setting():
    change_remote_cc_from_client_to_server()
    global server_iqn 
    print("iscsi server를 세팅해보겠습니다~")
    cc("lsblk")
    disk = input("위에 결과에서 확인한 공유할 물리 디스크를 입력해주세요 ex. /dev/sdb :")
    domain = input("iSCSI 도메인을 입력해주세요. 이 값은 고유해야합니다. ex. 2025-11.local.r1:disk1 :")
    server_iqn = f"iqn.{domain}"
    
    commands = f"""
cat <<EOF > ~/target_server_setting.exp
#!/usr/bin/expect -f
spawn targetcli

expect "/> " 
send "backstores/block create name=disk1 dev={disk}\\r"

expect "/> "
send "iscsi/ create {server_iqn}\\r"

expect "/> "
send "iscsi/{server_iqn}/tpg1/luns create /backstores/block/disk1\\r"

expect "/> "
send "iscsi/{server_iqn}/tpg1/acls create {client_iqn}\\r"

expect "/> "
send "iscsi/{server_iqn}/tpg1/portals delete 0.0.0.0 3260\\r"

expect "/> "
send "iscsi/{server_iqn}/tpg1/portals create {server_ip}\\r"

expect "/> "
send "exit\\r"
expect eof
EOF

chmod +x ~/target_server_setting.exp 
expect ~/target_server_setting.exp
"""
    cc(commands)
    print("targetcli 설정이 완료됨.")





def iscsi_server_restart():
    print("iscsi 서버를 다시 실행하겠습니다")
    cmd = f"""
systemctl enable target
systemctl restart target
"""
    cc(cmd)

def iscsi_client_login_set_servers_iscsi():
    change_remote_cc_from_server_to_client()
    print("로그인 하겠습니다.")
    cc("sudo systemctl enable --now iscsid")
    cc(f"iscsiadm -m discovery -t st -p {server_ip}")
    cc(f"iscsiadm -m node -T {server_iqn} -p {server_ip}  -l")
    cc("lsblk")
    
def iscsi() :
   # iscsi_server_install()
    iscsi_server_targetcli_start()
    iscsi_client_install_start()
    iscsi_server_setting()
    iscsi_server_restart()
    iscsi_client_login_set_servers_iscsi()

iscsi()



targetcli 시작합니다.
Created symlink /etc/systemd/system/multi-user.target.wants/target.service → /usr/lib/systemd/system/target.service.



client가 Rocky인 경우 R , Ubuntu인 경우 U를 입력해주세요 R


client : Rocky
Extra Packages for Enterprise Linux 9 - x86_64   15 kB/s | 9.3 kB     00:00    
Extra Packages for Enterprise Linux 9 - x86_64  6.7 MB/s |  20 MB     00:02    
Remi's Modular repository for Enterprise Linux  2.9 kB/s | 3.5 kB     00:01    
Remi's Modular repository for Enterprise Linux  260 kB/s | 910 kB     00:03    
Safe Remi's RPM repository for Enterprise Linux 2.5 kB/s | 3.0 kB     00:01    
Safe Remi's RPM repository for Enterprise Linux 398 kB/s | 1.4 MB     00:03    
Rocky Linux 9 - BaseOS                          6.3 kB/s | 4.1 kB     00:00    
Rocky Linux 9 - AppStream                       7.6 kB/s | 4.5 kB     00:00    
Rocky Linux 9 - CRB                             7.8 kB/s | 4.5 kB     00:00    
Rocky Linux 9 - Extras                          4.9 kB/s | 2.9 kB     00:00    
Dependencies resolved.
 Package                        Arch   Version                     Repo    Size
Installing:
 iscsi-initiator-utils          x86_64 6.2.1.9-1.gita65a472.el9    ba

위에서 확인한 client의 iqn을 복사해주세요. ex. iqn.1994-05.com.redhat:30eb3450bd10 : iqn.1994-05.com.redhat:90f3ea491d36


iscsi server를 세팅해보겠습니다~
NAME        MAJ:MIN RM  SIZE RO TYPE MOUNTPOINTS
sda           8:0    0  102M  0 disk 
sdb           8:16   0  102M  0 disk 
sdc           8:32   0  102M  0 disk 
sr0          11:0    1 1024M  0 rom  
nvme0n1     259:0    0   20G  0 disk 
├─nvme0n1p1 259:1    0    1G  0 part /boot
└─nvme0n1p2 259:2    0   19G  0 part 
  ├─rl-root 253:0    0   17G  0 lvm  /
  └─rl-swap 253:1    0    2G  0 lvm  [SWAP]



위에 결과에서 확인한 공유할 물리 디스크를 입력해주세요 ex. /dev/sdb : /dev/sdb
iSCSI 도메인을 입력해주세요. ex. 2025-11.local.r1:disk1 : 2025-11.local.r1:disk1


spawn targetcli
targetcli shell version 2.1.57
Copyright 2011-2013 by Datera, Inc and others.
For help on commands, type 'help'.

/> backstores/block create name=disk1 dev=/dev/sdb
Created block storage object disk1 using /dev/sdb.
/> iscsi/ create iqn.2025-11.local.r1:disk1
Created target iqn.2025-11.local.r1:disk1.
Created TPG 1.
Global pref auto_add_default_portal=true
Created default portal listening on all IPs (0.0.0.0), port 3260.
/> iscsi/iqn.2025-11.local.r1:disk1/tpg1/luns create /backstores/block/disk1
Created LUN 0.
/> iscsi/iqn.2025-11.local.r1:disk1/tpg1/acls create iqn.1994-05.com.redhat:90f3ea491d36
Created Node ACL for iqn.1994-05.com.redhat:90f3ea491d36
Created mapped LUN 0.
/> iscsi/iqn.2025-11.local.r1:disk1/tpg1/portals delete 0.0.0.0 3260
Deleted network portal 0.0.0.0:3260
/> iscsi/iqn.2025-11.local.r1:disk1/tpg1/portals create 172.16.16.122
Using default IP port 3260
Created network portal 172.16.16.122:3260.
/> exit
Global pref auto_save_on_exit=true
Configurati